In [1]:
import json
import openai

In [2]:
client = openai.Client()

In [3]:
def saudation_by_hours(hours):
    if 5 <= hours < 12:
        return json.dumps({"Saudação":"Bom dia!"})
    elif 12 <= hours < 18:
        return json.dumps({"Saudação":"Boa tarde!"})
    else:
        return json.dumps({"Saudação":"Boa noite!"})


In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudation_by_hours",
            "description": "Retorna uma saudação dependendo da hora do dia.",
            "parameters": {
                "type": "object",
                "properties": {
                    "hours": {
                        "type": "integer",
                        "description": "Hora do dia em formato 24h)"
                    }
                },
                "required": ["hours"]
            }
        }

    }
]

In [5]:
function1 = {
    "saudation_by_hours": saudation_by_hours
}

In [6]:
message = [{"role": "user", "content": "Qual a saudação para 10h?"}]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=message,
    tools=tools,
    tool_choice="auto"
)

In [7]:
message_reply = response.choices[0].message
message_reply

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wsnmvwk2ZaqjPYPoEACEunnM', function=Function(arguments='{"hours":10}', name='saudation_by_hours'), type='function')])

In [8]:
tool_calls = message_reply.tool_calls
tool_calls


[ChatCompletionMessageToolCall(id='call_wsnmvwk2ZaqjPYPoEACEunnM', function=Function(arguments='{"hours":10}', name='saudation_by_hours'), type='function')]

In [9]:
if tool_calls:
    message.append(message_reply)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = function1[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hours = function_args.get("hours")
        )

        message.append(
            {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
            }
        )

    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages = message
    )
    

mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='A saudação para às 10h é: "Bom dia!"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)